# Chianson Siu

## Directions
Your tool (and thus, geoprocessing service) needs to take the following parameters as inputs from the user, at a minimum:
orig_shapefile (to be general, assume this is a feature layer)
- infield
- outfield
- reclasstable
- notfoundvalue (a single number--assume it is a double)

Note: You can assume that the values in the attribute column of orig_shapefile specified by infield are **doubles** (numbers).
Your tool outputs/returns a Feature Class with both the original data from orig_shapefile and a newly added reclassified attribute called outfield whose values are found from the value of infield using the reclasstable (or if that value is not within the ranges of reclasstable, it is reclassified to notfoundvalue.)  

In [1]:
# import system paths and append directories to the ArcGIS loaction on local computer
import sys
import os
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')

# import arcpy package and set the working directory
import arcpy
workspace = "C:\\Users\\cjms2\\geog458\\lab_2\\"
arcpy.env.workspace = workspace

In [2]:
# Input parameters from the client.
# desired shapefile to reclassify
inFile = workspace + "King\\King.shp" 

# desired field in file to reclassify
inField = "POP2012" 

# table to reclass shapefile
reclassTable = workspace + "ReclassTableExample.dbf" 

# client's default value for inField values outside the range of reclass table
notFoundValue = 0 

In [3]:
# intialize empty list of lower bound values from reclass table
lowerBoundVal = []

# intialize empty list of upper bound values from reclass table
upperBoundVal = []

# intialize empty list of reclass values from reclass table
reclassVal = []

In [4]:
# populate each lower bound, uppder bound, and reclass value lists with their
# corresponding column values from the reclass table.
reclassSearch = arcpy.da.SearchCursor(reclassTable, ["lowerbound", "upperbound", "value"])
for row in reclassSearch:
    lowerBoundVal.append(row[0])
    upperBoundVal.append(row[1])
    reclassVal.append(row[2])
del row
del reclassSearch

In [5]:
# create output shapefile from input shapefile
outClass = None
if(os.path.exists(workspace + "result\\output.shp")):
    outClass = workspace + "result\\output.shp"
else:
    outClass = arcpy.CopyFeatures_management(inFile, workspace + "result\\output.shp")

In [148]:
# add 'outfield' attribute to the output shapefile
arcpy.AddField_management(outClass, "outfield", "DOUBLE")

<Result 'C:\\Users\\cjms2\\geog458\\lab_2\\result\\output.shp'>

In [6]:
# checkRange takes a lowerbound value, an uppder bound value, and number to be
# compared to each value. Returns true if num is between or equal to the upper
# or lower bound, else returns fase. 
# @param low: double, lower bound of comparison range
# @param high: double, upper bound of comparison range
# @param num: double, number to compared
# @return: boolean result if num is within the range of params 'low' and 'high'
def checkRange(low, high, num):
    return ((num <= high) & (num >= low))

In [7]:
# search cursor to search the input shapefile
inFileSearch = arcpy.da.SearchCursor(inFile, inField)
#inSearch.next() # move search cursor to first row

# update cursor to update the 'outfield' in the output shapefile with
# either values from the reclass table or notFoundValue from client
outUpdate = arcpy.da.UpdateCursor(outClass, "outfield")

# searches each value of the inField from input shapefile and checks if it
# is within range of the lower and upper bound values from the reclass table.
# This test is inclusive of upper and lower values. If the inField value
# is within the range, the reclass value from the reclass table is assigned
# to the tuple's 'outfield' attribute, else the notFoundValue is assigned.
for row in outUpdate:
    inSearch.next() # move search cursor to next row in input shapefile
    reclassResult = notFoundValue # default to notFoundValue
    
    # compares values in the lowerBoundVal and upperBoundVal list to find the
    # first occurence where the inField value is within their range.
    for i in range(0,len(reclassVal)):
        if (checkRange(lowerBoundVal[i], upperBoundVal[i], inFileSearch[0]) == True):
            reclassResult = reclassVal[i]
            break # first occurence found, exit nested loop
    row[0] = reclassResult # reclass outfield value
    outUpdate.updateRow(row) # update tuple with reclassified value

# delete row objects to remove locks
del row

# delete search and update cursors to remove locks
del inSearch
del outUpdate

In [115]:
temp = [1, 2, 5, 10, 20, 25, 60, 80, 90, 110, 112, 120, 140, 678, 3096]

In [119]:
def binarySearch(lowerBounds, num):
    midpoint = len(lowerBounds) / 2
    
#     if(num in lowerBounds): # num is contained in list
#         return lowerBounds.index(num)
    
    if(num >= lowerBounds[len(lowerBounds)-1]): # num is >= largest number in lower bound list
        return len(lowerBounds) - 1
    
    elif(num < lowerBounds[0]): # num is less than smallest number in lower bound list
        return -1
    
    elif(len(lowerBounds) == 2): # num is bounded bewteen the last two numbers in the lower bound list
        if(num in lowerBounds): # num is contained in the remaining two numbers
            return temp.index(lowerBounds[lowerBounds.index(num)])
        else:
            return temp.index(lowerBounds[0])
    
    elif(lowerBounds[midpoint] <= num): # num is >= the value in the middle of the lower bound list
        return binarySearch(lowerBounds[midpoint:len(lowerBounds)], num) # recursive call on top half of list
    
    else: # num is less than the value in the middle of the list
        return binarySearch(lowerBounds[0:(midpoint + 1)], num) # recursive call on bottom half of list
   
        

In [121]:
print binarySearch(temp, 678)

13
